In [1]:
import pandas as pd
import json
import gzip


In [2]:
file = "/home/marc/inters/output_test2/05_Annotation/Nirvana/snvs/Annotation.json.gz"

In [3]:

header = ''
positions = []
genes = []
is_header_line = True
is_position_line = False
is_gene_line = False
gene_section_line = '],"genes":['
end_line = ']}'

with gzip.open(file, 'rt') as f:
    position_count = 0
    gene_count = 0
    for line in f:
        trim_line = line.strip()
        if is_header_line:
            ## only keep the "header" field content from the line
            header = trim_line[10:-14]
            is_header_line = False
            is_position_line = True
            continue
        if trim_line == gene_section_line:
            is_gene_line = True
            is_position_line = False
            continue
        elif trim_line == end_line:
            break
        else:
            if is_position_line:
                ## remove the trailing ',' if there is
                positions.append(trim_line.rstrip(','))
                position_count += 1
            if is_gene_line:
                ## remove the trailing ',' if there is
                genes.append(trim_line.rstrip(','))
                gene_count += 1

print ('number of positions:', position_count)
print ('number of genes:', gene_count)


number of positions: 6557
number of genes: 313


In [20]:
mylist = []
for gene in genes:
    position_dict = json.loads(gene)
    mylist.append(position_dict)
genedf = pd.DataFrame(mylist)

In [21]:
genedf

,name,gnomAD,omim,clingenDosageSensitivityMap,clingenGeneValidity
0,AC099805.1,"{'pLi': 0.104, 'pRec': 0.604, 'pNull': 0.292, ...",NaN,NaN,NaN
1,AC100868.1,"{'pLi': 5.68e-12, 'pRec': 0.114, 'pNull': 0.88...",NaN,NaN,NaN
2,AC105001.2,"{'pLi': 3.48e-07, 'pRec': 0.571, 'pNull': 0.42...",NaN,NaN,NaN
3,AC110275.1,"{'pLi': 0.846, 'pRec': 0.154, 'pNull': 0.00010...",NaN,NaN,NaN
4,ADAM18,"{'pLi': 6.88e-35, 'pRec': 3.53e-07, 'pNull': 1...","[{'mimNumber': 619495, 'geneName': 'ADAM metal...",NaN,NaN
...,...,...,...,...,...
308,ZNF696,"{'pLi': 0.207, 'pRec': 0.755, 'pNull': 0.038, ...",NaN,NaN,NaN
309,ZNF703,"{'pLi': 0.356, 'pRec': 0.632, 'pNull': 0.0113,...","[{'mimNumber': 617045, 'geneName': 'Zinc finge...",NaN,NaN
310,ZNF704,"{'pLi': 0.995, 'pRec': 0.00468, 'pNull': 8.22e...",NaN,NaN,NaN
311,ZNF706,"{'pLi': 0.702, 'pRec': 0.283, 'pNull': 0.0146,...","[{'mimNumber': 619526, 'geneName': 'Zinc finge...",NaN,NaN


In [30]:
mylist = []
for position in positions:
    position_dict = json.loads(position)
    mylist.append(position_dict)
df = pd.DataFrame(mylist)
variants_df = df['variants'].apply(pd.Series)
variants_df2 = variants_df[0].apply(pd.Series)
final_df = pd.concat([df, variants_df2], axis=1).drop(['variants', 'samples'], axis=1)



In [5]:
final_df['transcripts'][30]

[{'transcript': 'ENST00000320901.3',
  'source': 'Ensembl',
  'bioType': 'protein_coding',
  'geneId': 'ENSG00000176269',
  'hgnc': 'OR4F21',
  'consequence': ['upstream_gene_variant'],
  'isCanonical': True,
  'proteinId': 'ENSP00000318878.3'},
 {'transcript': 'NM_001005504.1',
  'source': 'RefSeq',
  'bioType': 'protein_coding',
  'geneId': '441308',
  'hgnc': 'OR4F21',
  'consequence': ['upstream_gene_variant'],
  'isCanonical': True,
  'proteinId': 'NP_001005504.1'}]

In [27]:
final_df = pd.read_csv('/home/marc/inters/output_test2/05_Annotation/Nirvana/snvs/Annotation.tsv', delimiter='\t')


In [28]:
final_df

,chromosome,position,refAllele,altAlleles,quality,filters,fisherStrandBias,mappingQuality,cytogeneticBand,vid,...,transcripts,dbsnp,gnomad,topmed,globalAllele,regulatoryRegions,inLowComplexityRegion,oneKg,revel,clinvar
0,chr8,116079,G,['A'],301.90,['PASS'],7.799,60.00,8p23.3,8-116079-G-A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr8,116701,A,['G'],107.29,['PASS'],0.000,60.00,8p23.3,8-116701-A-G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr8,116895,A,['G'],161.28,['SOR3'],0.000,60.00,8p23.3,8-116895-A-G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr8,160552,G,['A'],36.29,['PASS'],0.000,59.25,8p23.3,8-160552-G-A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr8,160608,A,['C'],976.97,['PASS'],0.000,60.00,8p23.3,8-160608-A-C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6552,chr8,146200492,G,['A'],1241.95,['PASS'],0.000,60.00,NaN,8-146200492-G-A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6553,chr8,146203412,A,['G'],8533.95,['PASS'],3.114,60.00,NaN,8-146203412-A-G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6554,chr8,146220094,C,['T'],2706.95,['PASS'],0.000,60.00,NaN,8-146220094-C-T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6555,chr8,146229040,T,['G'],616.95,['PASS'],0.000,60.00,NaN,8-146229040-T-G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:


# Define the gene symbol you want to search for
target_gene_symbol = 'P53'
# final_df = pd.read_csv('/home/marc/inters/output_test2/05_Annotation/Nirvana/snvs/Annotation.tsv', delimiter='\t')

# Function to filter rows based on the target gene symbol
def filter_by_gene_symbol(row):
    if isinstance(row, list):
        for d in row:
            if isinstance(d, dict) and d.get('hgnc') == target_gene_symbol:
                return True
    return False

# Apply the filter function to the DataFrame
filtered_df = final_df[final_df['transcripts'].apply(filter_by_gene_symbol)]

# Print the filtered DataFrame
# print(filtered_df)

def is_novel(row):
    return pd.isna(row['dbsnp'])

total_rows = len(filtered_df)
total_variants = total_rows
variants_with_annotations = total_rows - filtered_df['dbsnp'].isna().sum()
novel_variants = total_variants - variants_with_annotations
percentage_novel = (novel_variants / total_rows) * 100
# Print the statistics
print()
print(f"#--------------------{target_gene_symbol}--------------------#")
print(f"Total Variants: {total_variants}")
print(f"Variants with Annotations: {variants_with_annotations}")
print(f"Variants without dbSNP Annotations: {novel_variants}")
print(f"Percentage of Novel Variants: {percentage_novel} %")

Total Variants: 0
Variants with Annotations: 0
Variants without dbSNP Annotations: 0
Percentage of Novel Variants: nan %


/home/marc/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in long_scalars


In [35]:
if not filtered_df.empty:
    pass
else:
    print("empty")

empty
